<a href="https://colab.research.google.com/github/gvanathip/classification/blob/main/remove_multicollinearity_with_NetworkX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset and Clean Process :
https://www.kaggle.com/code/iyet1killer/classification-on-categorical-data-part-1-sklearn/notebook

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('in-vehicle-coupon-recommendation.csv')

In [ ]:
df.head()

In [ ]:
# Ultimate Data Processing Function
def data_cleaning(data=df):
    clean_df = df.copy()

    clean_df.drop(columns=['car'], inplace=True)
    na_columns = ['Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20', 'Restaurant20To50']
    clean_df.drop(columns=['toCoupon_GEQ5min'], inplace=True)
    frequency_map = {
        'never': 0,
        'less1': 1,
        '1~3': 2,
        '4~8': 3,
        'gt8': 4}
    age_map = {
        'below21': 0,
        '21': 1,
        '26': 2,
        '31': 3,
        '36': 4,
        '41': 5,
        '46': 6,
        '50plus': 7}
    income_map = {
        'Less than $12500': 0,
        '$12500 - $24999': 1,
        '$25000 - $37499': 2,
        '$37500 - $49999': 3,
        '$50000 - $62499': 4,
        '$62500 - $74999': 5,
        '$75000 - $87499': 6,
        '$87500 - $99999': 7,
        '$100000 or More': 8}
    frequency_cols = ['Restaurant20To50', 'RestaurantLessThan20',
                      'CarryAway', 'CoffeeHouse', 'Bar']
    for col in frequency_cols:
        clean_df[col] = clean_df[col].map(frequency_map)
    clean_df.age = clean_df.age.map(age_map)
    clean_df.income = clean_df.income.map(income_map)
    clean_df.drop(columns=['direction_opp'], inplace=True)
    clean_df['distance'] = None
    clean_df.loc[clean_df['toCoupon_GEQ15min'] == 0, 'distance'] = 0
    clean_df.loc[(clean_df['toCoupon_GEQ15min'] == 1) & \
                 (clean_df['toCoupon_GEQ25min'] == 0), 'distance'] = 1
    clean_df.loc[clean_df['toCoupon_GEQ25min'] == 1, 'distance'] = 2
    clean_df.distance = clean_df.distance.astype('int64')
    clean_df.drop(columns=['toCoupon_GEQ15min', 'toCoupon_GEQ25min'], inplace=True)
    clean_df.has_children = clean_df.has_children.astype(str)
    clean_df.direction_same = clean_df.direction_same.astype(str)
    return clean_df

In [ ]:
df = data_cleaning(df)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.groupby(['Y'])['Y'].count()

In [ ]:
num_cols = sorted(list(df.select_dtypes(include="number").columns))
cate_cols = sorted(list(df.select_dtypes(exclude="number").columns))

In [ ]:
df[num_cols] = df[num_cols].fillna(0)
df[cate_cols] = df[cate_cols].fillna('-')

In [ ]:
cate_cols.append('Y')

In [ ]:
cor = df[num_cols].corr()
cor = pd.DataFrame(cor.unstack()).reset_index().rename(columns={"level_0": "x1", "level_1": "x2", 0: "r"})
cor['abs_r'] = abs(cor.r)
cor_y = dict(zip(cor[cor.x1=='Y'].x2,cor[cor.x1=='Y'].abs_r))

In [ ]:
cor.abs_r.hist()

In [ ]:
h_cor = cor[cor.abs_r > 0.2]
h_cor = h_cor[h_cor.x1 != h_cor.x2]

In [ ]:
h_cor

In [ ]:
import networkx as nx

In [ ]:
G = nx.from_pandas_edgelist(h_cor, 'x1', 'x2', ['abs_r'])
# G.remove_node('Y')
G = G.to_undirected()

In [ ]:
import matplotlib.pyplot as plt

nx.draw(G,with_labels=True, font_size=6)
plt.show()

In [ ]:
rnx = []
for i in sorted(set(h_cor.x1).union(set(h_cor.x2))):
#     print(i)

    if list(sorted(nx.node_connected_component(G,i))) in rnx:
        continue
    else:
#         print(list(sorted(nx.node_connected_component(G,i))))
        rnx.append(list(nx.node_connected_component(G,i)))
#         print('/////////////////////////')

In [ ]:
r_nx = []
for i in rnx:
    if sorted(i) in r_nx:
        continue
    else:
        r_nx.append(sorted(i))
#         print(sorted(i))

In [ ]:
g = 1
# ls = []
lss = dict()

for k in r_nx:
    lss[g] = k
    g = g+1

In [ ]:
h_cor_y = pd.DataFrame()
g = []
x = []
abs_cor_y = []

for ls in lss:
#     print(ls)
    for i in lss[ls]:
        g.append(ls)
        x.append(i)
        abs_cor_y.append(cor_y[i])
#             print(ls, i, cor_y[i])
#         print(lss[ls])
h_cor_y = pd.DataFrame(list(zip(g, x, abs_cor_y)),columns =['group', 'x', 'abs_cor_with_y'])

In [ ]:
h_cor_y['rank'] = h_cor_y.groupby('group')['abs_cor_with_y'].rank(ascending=False)
drop_h_cor_cols = list(set(h_cor_y[h_cor_y['rank'] > 1].x))

In [ ]:
h_cor_y

In [ ]:
drop_h_cor_cols

In [ ]:
from tqdm import tqdm

x1 = []
x2 = []
chi = []
pv = []
from scipy.stats import chi2_contingency
for i in tqdm(cate_cols):
    for j in cate_cols:
        if i != j:
#             print(i,':',j)
            x1.append(i)
            x2.append(j)
            CrosstabResult=pd.crosstab(df[i], df[j])
            try:
                ChiSqResult = chi2_contingency(CrosstabResult)
                chi.append(ChiSqResult[0])
                pv.append(ChiSqResult[1])
#                 if ChiSqResult[1] < 0.05:
#                     print(i,':',j)
#                     print('The P-Value of the ChiSq Test is:', ChiSqResult[1])
            except:
#                 chi.append(0)
                continue

In [ ]:
chi_df = pd.DataFrame(list(zip(x1, x2, chi, pv)),columns =['x1', 'x2', 'chi', 'pvalue'])

In [ ]:
# chi_df[chi_df.x1 == 'Y']

# sm_cor_y = dict(zip(chi_df[chi_df.x1 == 'Y'].x2,chi_df[chi_df.x1 == 'Y'].chi))
sm_cor_y = chi_df[chi_df.x1 == 'Y']

In [ ]:
h_cate_cols = chi_df[(chi_df.pvalue > 0.5) & (chi_df.x1 != 'Y') & (chi_df.x2 != 'Y') & (chi_df.x2 != chi_df.x1)]

In [ ]:
h_cate_cols

In [ ]:
G2 = nx.from_pandas_edgelist(h_cate_cols, 'x1', 'x2', ['pvalue'])
# G2.remove_node('Y')
G2 = G2.to_undirected()

In [ ]:
nx.draw(G2,with_labels=True, font_size=6)
plt.show()

In [ ]:
list(set(h_cate_cols.x1).union(set(h_cate_cols.x2)))

In [ ]:
cate_cols

for categorical variables most of them quite high correlation, so try to sub group with column name distance, for real world complex data for example 'price_tier1', 'price_tier2', 'price_tier3'

if it doesn't make sense, will not neccesary to execute.

In [ ]:
cnx = []
for i in sorted(set(h_cate_cols.x1).union(set(h_cate_cols.x2))):
#     print(i)

    if list(sorted(nx.node_connected_component(G2,i))) in cnx:
        continue
    else:
#         print(list(sorted(nx.node_connected_component(G2,i))))
        cnx.append(list(nx.node_connected_component(G2,i)))
#         print('/////////////////////////')

In [ ]:
c_nx = []
for i in cnx:
    if sorted(i) in c_nx:
        continue
    else:
        c_nx.append(sorted(i))
#         print(sorted(i))

In [ ]:
g = 1
# ls = []
lsc = dict()

for k in c_nx:
    lsc[g] = k
    g = g+1

In [ ]:
c_nx = [j for i in c_nx for j in i]

In [ ]:
def jaccard_binary(list1,list2):
    intersection = len(list(set(list1).intersection(set(list2))))
    union = len(list(set(list1).union(set(list2))))
    return float(intersection) / union

In [ ]:
x3 = []
x4 = []
sm = []
for i in c_nx:
    for j in c_nx:
        if i != j:
            # Convert the texts into TF-IDF vectors
            x3.append(i)
            x4.append(j)

            similarity = jaccard_binary([char for char in i],[char for char in j])
            sm.append(similarity)
#             print(i,j,similarity)

In [ ]:
sm_df = pd.DataFrame(list(zip(x3, x4, sm)),columns =['x1', 'x2', 'similarity'])

In [ ]:
sm_df.similarity.hist()

In [ ]:
h_sm_col_df = sm_df[sm_df.similarity > 0.6]

In [ ]:
h_sm_col_df

In [ ]:
G3 = nx.from_pandas_edgelist(h_sm_col_df, 'x1', 'x2', ['similarity'])
# G2.remove_node('Y')
G3 = G3.to_undirected()

In [ ]:
nx.draw(G3,with_labels=True, font_size=6)
plt.show()

In [ ]:
smnx = []
for i in sorted(set(h_sm_col_df.x1).union(set(h_sm_col_df.x2))):
#     print(i)

    if list(sorted(nx.node_connected_component(G3,i))) in smnx:
        continue
    else:
#         print(list(sorted(nx.node_connected_component(G3,i))))
        smnx.append(list(nx.node_connected_component(G3,i)))
#         print('/////////////////////////')

In [ ]:
sm_nx = []
for i in smnx:
    if sorted(i) in sm_nx:
        continue
    else:
        sm_nx.append(sorted(i))
#         print(sorted(i))

In [ ]:
g = 1
# ls = []
lssm = dict()

for k in sm_nx:
    lssm[g] = k
    g = g+1

In [ ]:
h_sm_cor_y = pd.DataFrame()
g = []
x = []
chi_cor_y = []
pv_cor_y = []

for ls in lssm:
#     print(ls)
    for i in lssm[ls]:
        g.append(ls)
        x.append(i)
        try:
            chi_cor_y.append(sm_cor_y[sm_cor_y.x2 == i].chi.values[0])
            pv_cor_y.append(sm_cor_y[sm_cor_y.x2 == i].pvalue.values[0])
        except:
            chi_cor_y.append(0)
            pv_cor_y.append(0)
#             print(ls, i, cor_y[i])
#         print(lss[ls])
h_sm_cor_y = pd.DataFrame(list(zip(g, x, chi_cor_y, pv_cor_y)),columns =['group', 'x', 'chi_with_y', 'pvalue_with_y'])

In [ ]:
h_sm_cor_y

In [ ]:
h_sm_cor_y['rank_chi'] = h_sm_cor_y.groupby('group')['chi_with_y'].rank(ascending=False)
h_sm_cor_y['rank_pv'] = h_sm_cor_y.groupby('group')['pvalue_with_y'].rank(ascending=True)
# drop_h_cor_cols = list(set(h_cor_y[h_cor_y['rank'] > 1].x))

In [ ]:
drop_h_sm_cor_cols = list(set(h_sm_cor_y[h_sm_cor_y['rank_chi'] > 1].x))

In [ ]:
drop_h_sm_cor_cols

In [ ]:
# for real world complex data, we found that we can group similarity category columns resonably
# drop_cor_cols = drop_h_cor_cols+drop_h_sm_cor_cols

In [ ]:
# for this example, category columns not nessesary to remove multicollinearity
drop_cor_cols = drop_h_cor_cols

In [ ]:
drop_cor_cols

In [ ]:
df = df.drop(columns=drop_cor_cols)

In [ ]:
df.info()